![logo](./img/LogoLine_horizon_C3S.png)

<br>

# Download Reanalysys: ERA5

### About


The notebook has the following outline:
* 1 - Download data from the CDS
* 2 - Hindcast data processing: calculate the reference climate mean
* 3 - Real-time forecasts: calculate seasonal forecast anomalies
* 4 - Visualize seasonal forecast monthly anomalies for a geographical subregion
  * 4.1 - Spatial maps
  * 4.2 - Time series of regional averages
* 5 - Reproduce C3S graphical products: compute 3-month anomalies

<br>

### Install packages

In [ ]:
# Install CDS API for downloading data from the CDS
!pip install cdsapi

In [ ]:
# Install cfgrib to enable us to read GRIB format files
#!conda install -c conda-forge cfgrib -y

In [1]:
# Miscellaneous operating system interfaces
import os

# CDS API
import cdsapi

# To map GRIB files to NetCDF Common Data Model
import cfgrib

# Libraries for working with multi-dimensional arrays
import numpy as np
import xarray as xr
import pandas as pd
import scipy

# Libraries for plotting and geospatial data visualisation
import matplotlib.path as mpath
import matplotlib.pyplot as plt

import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cartopy.feature as cfeature

# To work with data labels in dictionary format
from collections import OrderedDict

# Date and time related libraries
from dateutil.relativedelta import relativedelta
from calendar import monthrange
import datetime

# Interactive HTML widgets
import ipywidgets as widgets

# Disable warnings for data download via API
import urllib3 
urllib3.disable_warnings()

<hr>

## 1. Request data from the CDS programmatically with the CDS API

The first step is to request data from the Climate Data Store programmatically with the help of the CDS API. Let us make use of the option to manually set the CDS API credentials. First, you have to define two variables: `URL` and `KEY` which build together your CDS API key. Below, you have to replace the `#########` with your personal CDS key. Please find [here](https://cds.climate.copernicus.eu/api-how-to) your personal CDS key.

In [4]:
URL = 'https://cds.climate.copernicus.eu/api'  #https://cds.climate.copernicus.eu/api'
KEY = '7c7e2544-915e-4f95-b994-8a949747cd5d'

Here we specify a data directory in which we will download our data and all output files that we will generate:

In [5]:
DATADIR = 'C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/reanalysis'

C:/Users/apedregal/Documents/inventWater_docs/Modelling/Seasonal forecasts/reanalysis


In [6]:
c = cdsapi.Client(url=URL, key=KEY)

2025-02-12 21:02:27,359 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-02-12 21:02:27,360 WARNING [2024-06-16T00:00:00] CDS API syntax is changed and some keys or parameter names may have also changed. To avoid requests failing, please use the "Show API request code" tool on the dataset Download Form to check you are using the correct syntax for your API request.


### Download precipitation

In [12]:
variable = 'total_precipitation'
var_short = 'tp'

In [9]:
for year in np.arange(2023,2025):
    c.retrieve(
    'reanalysis-era5-single-levels',
    {
        'product_type': 'reanalysis',
        'format': 'grib',
        'variable': variable,
        'year': str(year),
        'month': [
            '01', '02', '03',
            '04', '05', '06',
            '07', '08', '09',
            '10', '11', '12',
        ],
        'day': [
            '01', '02', '03',
            '04', '05', '06',
            '07', '08', '09',
            '10', '11', '12',
            '13', '14', '15',
            '16', '17', '18',
            '19', '20', '21',
            '22', '23', '24',
            '25', '26', '27',
            '28', '29', '30',
            '31',
        ],
        'time': [
            '00:00', '01:00', '02:00',
            '03:00', '04:00', '05:00',
            '06:00', '07:00', '08:00',
            '09:00', '10:00', '11:00',
            '12:00', '13:00', '14:00',
            '15:00', '16:00', '17:00',
            '18:00', '19:00', '20:00',
            '21:00', '22:00', '23:00',
        ],
        'area': [
            43, 2, 42,
            3,
        ],
    },
    f'{DATADIR}/ecmwf_era5_' + str(year) +'_hourly_' + var_short + '.grib')
    

2025-02-12 21:06:01,135 INFO Request ID is 88e87e08-b84d-4c05-a672-f784609fe364
2025-02-12 21:06:01,222 INFO status has been updated to accepted
2025-02-12 21:06:06,204 INFO status has been updated to running
2025-02-12 21:18:21,158 INFO status has been updated to successful


b4a6615f6f71320a670c5aa5073901ce.grib:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

2025-02-12 21:18:22,805 INFO Request ID is 474aa40e-591d-496d-9146-3497b90fb6c5
2025-02-12 21:18:22,897 INFO status has been updated to accepted
2025-02-12 21:18:31,397 INFO status has been updated to running
2025-02-12 21:18:36,531 INFO status has been updated to successful


e042693b0e02e38f156c1e3b9d8dfc58.grib:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

### Download air temperature

In [13]:
variable = '2m_temperature'
var_short = 't2m'

In [14]:
for year in np.arange(2023,2025):
    c.retrieve(
    'reanalysis-era5-single-levels',
    {
        'product_type': 'reanalysis',
        'format': 'grib',
        'variable': variable,
        'year': str(year),
        'month': [
            '01', '02', '03',
            '04', '05', '06',
            '07', '08', '09',
            '10', '11', '12',
        ],
        'day': [
            '01', '02', '03',
            '04', '05', '06',
            '07', '08', '09',
            '10', '11', '12',
            '13', '14', '15',
            '16', '17', '18',
            '19', '20', '21',
            '22', '23', '24',
            '25', '26', '27',
            '28', '29', '30',
            '31',
        ],
        'time': [
            '00:00', '01:00', '02:00',
            '03:00', '04:00', '05:00',
            '06:00', '07:00', '08:00',
            '09:00', '10:00', '11:00',
            '12:00', '13:00', '14:00',
            '15:00', '16:00', '17:00',
            '18:00', '19:00', '20:00',
            '21:00', '22:00', '23:00',
        ],
        'area': [
            43, 2, 42,
            3,
        ],
    },
    f'{DATADIR}/ecmwf_era5_' + str(year) +'_hourly_' + var_short + '.grib')
    

2025-02-12 21:19:21,292 INFO Request ID is 6e8b4d49-16af-47c5-9ba8-b45564c0a643
2025-02-12 21:19:21,351 INFO status has been updated to accepted
2025-02-12 21:19:26,549 INFO status has been updated to running
2025-02-12 21:19:30,039 INFO status has been updated to accepted
2025-02-12 21:19:54,396 INFO status has been updated to running
2025-02-12 21:29:41,830 INFO status has been updated to successful


43e72c5f2e6308f4b0421f5ba77a7e6b.grib:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

2025-02-12 21:29:44,634 INFO Request ID is d11a6bdc-751e-4dad-9ac7-865d173d5227
2025-02-12 21:29:44,819 INFO status has been updated to accepted
2025-02-12 21:30:00,363 INFO status has been updated to successful


c9424b784e09252ed8bcf9e9ea7f36eb.grib:   0%|          | 0.00/1.32M [00:00<?, ?B/s]